In [30]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold, RepeatedKFold;

In [31]:
# load in train and test datasets
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

#Store our passenger ID for easy access
PassengerId = test['PassengerId']

train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [32]:
full_data = [train, test]

train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)

train['Has_Cabin'] = train['Cabin'].apply(lambda x:0 if type(x) == float else 1)
test['Has_Cabin'] = test['Cabin'].apply(lambda x:0 if type(x) == float else 1)

#Feature engineering
#Create new feature FamilySize as a combination of sibsp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
# Create new feature IsAlone from FamilySize
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone' ] = 1

for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
for dataset in full_data:
    dataset['Fare']= dataset['Fare'].fillna(train['Fare'].median())
train['CategoricalFare'] = pd.qcut(train['Fare'],4)

for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
train['CategoricalAge'] = pd.cut(train['Age'], 5)
# Define the function to extract titles from passengers' names 
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
    
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady','Countess','Capt','Col','Don',
                                                 'Dr','Major','Rev','Sir','Jonkheer','Dona'],'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    dataset['Sex'] = dataset['Sex'].map( {'female':0, 'male':1}).astype(int)
    title_mapping = {"Mr":1 ,"Miss":2, "Mrs":3 ,"Master": 4 ,"Rare":5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    dataset['Embarked'] = dataset['Embarked'].map({'S':0, 'C':1 ,'Q':2}).astype(int)
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare']  = 0
    dataset.loc[ (dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[ (dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[ (dataset['Fare'] > 31), 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    dataset.loc[ dataset['Age'] <= 16, 'Age' ] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;
    
# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
test  = test.drop(drop_elements, axis = 1)
    

In [33]:
train.head(5)

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,0,3,1,1,0,0,0,23,0,2,0,1
1,1,1,0,2,0,3,1,51,1,2,0,3
2,1,3,0,1,0,1,0,22,0,1,1,2
3,1,1,0,2,0,3,0,44,1,2,0,3
4,0,3,1,2,0,1,0,24,0,1,1,1


In [34]:
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self, x, y):
        return self.clf.fit(x,y)
    
    def feature_importances(self, x, y):
        print(self.clf.fit(x,y).feature_importances_)


In [35]:
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 226
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True ,random_state=SEED) #K-交叉验证

#kf = RepeatedKFold(n_splits=NFOLDS, n_repeats =2 ,random_state=SEED) #K-交叉重复验证
def get_oof(clf, x_train, y_train, x_test):   #x_train 训练集 y_train 训练集标签 x_test测试集
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i,(train_index, test_index) in enumerate(kf.split(x_train)): #train_index 被分到训练集中的index  同理test_index
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
        y_te = y_train[test_index]
        clf.train(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te) # 用验证数据集验证训练集训练好的模型
        
        #oof_test_skf[i, :] = clf.predict(x_test)  # 用测试集测试
        print("RF K-fold: %d accuracy score is %f " % (i,accuracy_score(y_te.reshape(-1),clf.predict(x_te).reshape(-1))))
    #oof_test[:] = oof_test_skf.mean(axis=0)
    oof_test[:] = clf.predict(x_test)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)   

In [36]:
# Put in our parameters for said classifiers
# Forest note:
# the main parameters for these "forest" algorithm is 'n_estimators' and 'max_features'
# Random Forest parameters
rf0_params = {
    'n_jobs': -1,                # the number of jobs to run in parallel for both fit and predict
    'n_estimators': 900,         # The number of trees in the forest
     #'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 35,
    'min_samples_leaf': 1,
    'max_features' : None,    # the max number of features to consider when looking for the best split
    'verbose': 0
}
rf1_params = {
    'n_jobs': -1,                # the number of jobs to run in parallel for both fit and predict
    'n_estimators': 900,         # The number of trees in the forest
     #'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 35,
    'min_samples_leaf': 1,
    'max_features' : 2,    # the max number of features to consider when looking for the best split
    'verbose': 0
}

rf2_params = {
    'n_jobs': -1,                # the number of jobs to run in parallel for both fit and predict
    'n_estimators': 900,         # The number of trees in the forest
     #'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 35,
    'min_samples_leaf': 1,
    'max_features' : 'sqrt',    # the max number of features to consider when looking for the best split
    'verbose': 0
}

In [37]:
# Create 5 objects that represent our 4 models
rf0 = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf0_params)
rf1 = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf1_params)
rf2 = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf2_params)

In [38]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['Survived'].ravel()
train = train.drop(['Survived'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creats an array of the test data

In [39]:
# Create our OOF train and test predictions. These base results will be used as new features
rf_oof_train0, rf_oof_test0 = get_oof(rf0,x_train, y_train, x_test) # Random Forest
rf_oof_train1, rf_oof_test1 = get_oof(rf1,x_train, y_train, x_test)
rf_oof_train2, rf_oof_test2 = get_oof(rf2,x_train, y_train, x_test)
print("Training is complete")

RF K-fold: 0 accuracy score is 0.759777 
RF K-fold: 1 accuracy score is 0.764045 
RF K-fold: 2 accuracy score is 0.792135 
RF K-fold: 3 accuracy score is 0.831461 
RF K-fold: 4 accuracy score is 0.820225 
RF K-fold: 0 accuracy score is 0.754190 
RF K-fold: 1 accuracy score is 0.775281 
RF K-fold: 2 accuracy score is 0.803371 
RF K-fold: 3 accuracy score is 0.837079 
RF K-fold: 4 accuracy score is 0.825843 
RF K-fold: 0 accuracy score is 0.765363 
RF K-fold: 1 accuracy score is 0.775281 
RF K-fold: 2 accuracy score is 0.797753 
RF K-fold: 3 accuracy score is 0.825843 
RF K-fold: 4 accuracy score is 0.831461 
Training is complete


In [40]:
#print(rf_oof_train.reshape(-1))
print(accuracy_score(y_train,rf0.predict(x_train)))
print(accuracy_score(y_train,rf1.predict(x_train)))
print(accuracy_score(y_train,rf2.predict(x_train)))

0.94051627385
0.941638608305
0.942760942761


In [12]:
#直接某个基学习器
#print(rf_oof_test.round())
#StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
#                           'Survived': rf_oof_test.round().reshape(-1).astype(np.int8)})
#StackingSubmission.to_csv("RFSubmission.csv", index=False)